In [106]:
import pandas as pd
import numpy as np
import pickle
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
df = pd.read_csv('../Data/Interim/hydrated_Tweet200316.csv', index_col=0)

In [ ]:
# Clean tweets
df['CleanTweet'] = [re.sub("[^a-zA-Z0-9\s,.-_´&%'\":€$£!?']", '',  re.sub(' http\S+', '', re.sub('\s',' ', tw))).replace(u'\xa0', u' ').lower() if isinstance(tw, str) else '' for tw in df.tweet]

# Get bigrams
bigrams_tweets = [list(nltk.bigrams(tw)) for tw in df['CleanTweet']]

In [ ]:
with open('../Data/Processed/Bigram_tweets.pickle', 'wb') as handle:
    pickle.dump(bigrams_tweets, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
unique_chars = sorted(''.join(set(''.join(df['CleanTweet'])))) # Assume all characters are in tweets from df1

unique_bigrams = [x+y for x in unique_chars for y in unique_chars]

bigram_mapper = dict(zip(unique_bigrams, range(len(unique_bigrams))))

In [ ]:
Vector = CountVectorizer()

In [ ]:
X = Vector.fit_transform(df['CleanTweet'].values)

In [ ]:
X.shape

In [ ]:
lda = LatentDirichletAllocation(n_components=5, verbose = 2, n_jobs = 3)
lda.fit(X[:10000])

In [ ]:
topics = lda.transform(X[:10000]).argmax(axis = 1)

In [ ]:
y = np.bincount(topics)
ii = np.nonzero(y)[0]
dict(zip(ii,y[ii]))

In [ ]:
lda.score(X[:10000])

In [ ]:
from gensim.test.utils import common_corpus

In [ ]:
from gensim.models import LdaModel

In [ ]:
lda = LdaModel(X[:2000].todense(), num_topics=10)

In [1]:
import pickle

In [179]:
with open("../data/processed/embedding.pickle", "rb") as f:
    embedding = pickle.load(f)

In [84]:
from torch.nn.utils.rnn import pad_packed_sequence
from src.models.rvae import RecurrentVariationalAutoencoder
from src.models.rae import RecurrentAutoencoder
from src.models.rvae_words import RVAEWords
from src.models.common import *
from src.data.words import *
from src.data.characters import *

import torch

rvae_w, rvae_w_tinfo = get_trained_model(RVAEWords, training_info=True)

data = torch.load("../data/processed/200316_embedding.pkl")

num_samples = 10

ds = TwitterDataWords(data[-10:])
loader = get_loader(ds, batch_size=num_samples)

x = next(iter(loader))

output = rvae_w(x)

# Sample from observation model and pack, then pad
sample_packed = PackedSequence(output['px'].sample(), x.batch_sizes)
sample_padded, _ = pad_packed_sequence(sample_packed)

target_padded, _ = pad_packed_sequence(x)

# word x batch x embedding_dim
print(sample_padded.shape)

for i in range(num_samples):

    decoded_tweet = sample_padded[:, i, :]

torch.Size([40, 10, 300])


In [187]:
cse = []

for i in range(num_samples):
    print(actual_decode_similarity(sample_padded[:, i, :], target_padded[:, i, :], embedding))

(0.5129075773060322, 0.35068803895264866, 0.3410208935188737)
(0.5270840534142085, 0.3685850002935955, 0.3374059333518441)
(0.543899441404002, 0.3568235156791551, 0.4034492993815841)
(0.5090602698425452, 0.3469504117965698, 0.3434752716877078)
(0.5416954826740992, 0.3382833855492728, 0.32726740910620783)
(0.5203982701427058, 0.3447999915010051, 0.3455739531507516)
(0.5089935730485355, 0.3343326493221171, 0.37804772814587034)
(0.5297945118867434, 0.3161477589836487, 0.3923036544400154)
(0.550508975982666, 0.38519537895917894, 0.42148862313303737)
(0.6292392100606646, 0.4653415467057909, 0.4580013376048733)
